# Handling Loops

## Overview

One of the most common tasks is to accelerate/ parallelize (nested) for loops.
OpenMP offers worksharing constructs to facilitate this.

<div class="alert alert-block alert-warning"> <b>Warning:</b> We assume pleasingly parallel loops, i.e. loops in which no data dependencies between iterations exist. </div>

This loop fulfills the restriction (as long as `a` and `b` are not _aliased_)
```cpp
for (auto i = 1; i < N; ++i)
    a[i] = b[i] - b[i - 1];
```
This one doesn't
```cpp
for (auto i = 1; i < N; ++i)
    a[i] = a[i - 1];
```

If loops carry data dependencies, different approaches are possible:
* loop transformations to eliminate data dependencies (loop splitting/ fusion, loop remapping),
* data duplication (e.g. introducing source and destination arrays), or
* additional [synchronization](synchronization.ipynb).

## Manual Approach

Let's first load our custom cell magic and implement a serial baseline to be parallelized afterwards.

In [ ]:
%load_ext ice.magic

Our baseline code performs a *vector init*, i.e. sets each element of an array to a fixed value.
\
We add a print statement after the loop to prevent the compiler from optimizing the whole loop away.

In [ ]:
%%cpp -o code/loops/serial.cpp

constexpr auto N = 1024;
int vec[N];

for (auto i = 0; i < N; ++i)
    vec[i] = i;

std::cout << vec[N - 1] << std::endl;

Using only the tools we've learned so far already allows parallelization.
\
For this, we divide the loop iteration space into (almost) equal sized chunks.
\
Each thread then sets all the elements in its respective chunk.

In [ ]:
%%cpp_omp -o code/loops/manual.cpp

constexpr auto N = 1024;
int vec[N];

#pragma omp parallel
{
    int tid = omp_get_thread_num();                 //# get thread number
    int nt = omp_get_num_threads();                 //# get number of threads
    int per_thread = N / nt;                        //# elements per thread (rounded down)
    int remainder = N % nt;                         //# remaining elements (due to rounding)
    int lower  = tid * per_thread;                  //# start of this thread's chunk
    lower += tid < remainder ? tid : remainder;     //# modification for uneven chunk size
    int upper  = lower + per_thread;                //# end of this thread's chunk
    upper += tid < remainder ? 1 : 0;               //# modification for uneven chunk size

    for (int i = lower; i < upper; ++i)             //# loop over this thread's chunk
        vec[i] = i;                                 //# vector element initialization
}

std::cout << vec[N - 1] << std::endl;

## For Construct

While this style of coding works, it is quite cumbersome.
Using the OpenMP `for` construct instructs the compiler to do something similar ([OpenMP 5.1 - 2.11.4](https://www.openmp.org/spec-html/5.1/openmpsu48.html)).
This requires threads to be already existent, i.e. this constructs expects an enclosing parallel region.

<div class="alert alert-block alert-info"> <b>Note:</b> This automatically privatizes the loop iterator. </div>

In [ ]:
%%cpp_omp -o code/loops/for.cpp -v

constexpr auto N = 1024;
int vec[N];

#pragma omp parallel
{
#pragma omp for
    for (auto i = 0; i < N; ++i)
        vec[i] = i;
}

std::cout << vec[N - 1] << std::endl;

At the end of the region and implicit barrier is executed. It can be eliminated by adding the `nowait` clause which might lead to better resource utilization.

<div class="alert alert-block alert-warning"> <b>Warning:</b> Only do this when there are no data dependencies between subsequent loops. </div>

<div class="alert alert-block alert-info"> <b>Note:</b> Merging loops may lead to even better performance. </div>

In [ ]:
%%cpp_omp -o code/loops/nowait.cpp

constexpr auto N = 1024;
int vec[N];

#pragma omp parallel
{
#pragma omp for nowait
    for (auto i = 0; i < N / 2; ++i)
        vec[i] = i;

    //# no implicit barrier

#pragma omp for nowait
    for (auto i = N / 2; i < N; ++i)
        vec[i] = i;
}

std::cout << vec[N - 1] << std::endl;

In case of only a single loop parallel and for can be combined. This leads to even less code and also introduces only a single barrier.

In [ ]:
%%cpp_omp -o code/loops/parallel-for.cpp

constexpr auto N = 1024;
int vec[N];

#pragma omp parallel for
for (auto i = 0; i < N; ++i)
    vec[i] = i;

std::cout << vec[N - 1] << std::endl;

## Scheduling

In our example, we divided the iteration space into equal sized chunks.
OpenMP also supports this, as well as additional options ([OpenMP 5.1 - 2.11.4 - Table 2.5](https://www.openmp.org/spec-html/5.1/openmpsu48.html)):
|                         |                                                                 |
|-------------------------|-----------------------------------------------------------------|
| `static[, chunk_size]`  | fixed size chunks, static thread mapping                        |
| `dynamic[, chunk_size]` | fixed size chunks, dynamic (non-deterministic) thread mapping   |
| `guided[, chunk_size]`  | gradually smaller chunk sizes, dynamic thread mapping           |
| `auto`                  | implementation defined                                          |
| `runtime`               | schedule chosen at runtime via the `OMP_SCHEDULE` env. variable |

*static*, *dynamic* and *guided* support an optional chunk size - see below for details.

Let's start with a simple demo application introducing a severe load imbalance:
* Each loop iteration represents work that scales linearly with the loop iteration number. In other words, we wait for `i` ms where `i` is the loop iterator.
* We time time required for processing the whole loop.
* We deliberately chose a number of threads << number of iterations to allow for different scheduling options.

In [ ]:
%%cpp_omp -o code/loops/def-schedule.cpp -t

constexpr auto N = 128;

#pragma omp parallel for num_threads(8)
for (auto i = 0; i < N; ++i)
    std::this_thread::sleep_for(std::chrono::milliseconds(i));

### Static

Chunks are distributed statically in a round-robin fashion.
If chunk size is not defined, each threads gets exactly one chunk.

Possible execution for `schedule(static)`


<img src="img/loops/static.svg" alt="static" width="30%"/>

Alternative possiblity for `schedule(static)`


<img src="img/loops/static-alt.svg" alt="static-alternative" width="30%"/>

Possible execution for `schedule(static, 2)`


<img src="img/loops/static-2.svg" alt="static(2)" width="30%"/>

In [ ]:
%%cpp_omp -o code/loops/static.cpp -t

constexpr auto N = 128;

#pragma omp parallel for num_threads(8) schedule(static, 16)
for (auto i = 0; i < N; ++i)
    std::this_thread::sleep_for(std::chrono::milliseconds(i));

### Dynamic

Chunks are assigned to threads as soon as they become idle.
The chunk size is one unless specified.

Possible execution for `schedule(dynamic)`


<img src="img/loops/dynamic.svg" alt="dynamic" width="30%"/>

Possible execution for `schedule(dynamic, 2)`


<img src="img/loops/dynamic-2.svg" alt="dynamic(2)" width="30%"/>

In [ ]:
%%cpp_omp -o code/loops/dynamic.cpp -t

constexpr auto N = 128;

#pragma omp parallel for num_threads(8) schedule(dynamic, 4)
for (auto i = 0; i < N; ++i)
    std::this_thread::sleep_for(std::chrono::milliseconds(i));

### Guided

The chunk size gets gradually smaller until a smalles chunk size.
Its default value is 1 unless overwritten by specifying a chunk size.

Possible execution for `schedule(guided)`


<img src="img/loops/guided.svg" alt="guided" width="30%"/>

In [ ]:
%%cpp_omp -o code/loops/guided.cpp -t

constexpr auto N = 128;

#pragma omp parallel for num_threads(8) schedule(guided)
for (auto i = 0; i < N; ++i)
    std::this_thread::sleep_for(std::chrono::milliseconds(i));

### Runtime

The schedule is set at runtime via the environment variable `OMP_SCHEDULE` ([OpenMP 5.1 - 6.1](https://www.openmp.org/spec-html/5.1/openmpse58.html)).

Alternatively, the schedule to be used can be set by calling `omp_set_schedule(omp_sched_t kind, int chunk_size)` ([OpenMP 5.1 - 3.2.11](https://www.openmp.org/spec-html/5.1/openmpsu130.html)).
To query the runtime schedule, the `omp_get_schedule(omp_sched_t *kind, int *chunk_size)` function can be used ([OpenMP 5.1 - 3.2.12](https://www.openmp.org/spec-html/5.1/openmpsu131.html)).

In [ ]:
%%cpp_omp -o code/loops/runtime.cpp -t -e OMP_SCHEDULE=dynamic,12 -v

constexpr auto N = 128;

#pragma omp parallel for num_threads(8) schedule(runtime)
for (auto i = 0; i < N; ++i)
    std::this_thread::sleep_for(std::chrono::milliseconds(i));

In [ ]:
%%cpp_omp -o code/loops/print-schedule.cpp -t -e OMP_SCHEDULE=dynamic,12 -v

//# print schedule
omp_sched_t kind;
int chunk_size;
omp_get_schedule(&kind, &chunk_size);
std::cout << "Kind: " << kind << " ; Chunk Size: " << chunk_size << std::endl;

Note that `omp_sched_t` is an enum with the following (as well as additional) options
```cpp
  omp_sched_static  = 0x1, 
  omp_sched_dynamic = 0x2, 
  omp_sched_guided  = 0x3, 
  omp_sched_auto    = 0x4, 
```

### Summary

| schedule  | works well when ...                                                    |
|-----------|------------------------------------------------------------------------|
| `static`  | ... each loop iteration roughly takes the same amount of time          |
| `dynamic` | ... a strong load imbalance between loop iterations exists             |
| `guided`  | ... load imbalances exist, but performance benefits from larger chunks |
| `auto`    | ...                                                                    |
| `runtime` | ... different schedules are to be evaluated with a single binary       |

## Exercise: Stream Benchmark

<div class="alert alert-block alert-success"> <b>Exercise:</b> Investigate different schedules. </div>

Check the code for the stream benchmark application at [code/examples/stream.cpp](code/examples/stream.cpp) and the documentation in the [examples notebook](examples.ipynb#Stream-Benchmark).
For convenience, the cells for building and executing are copied below.
\
Parallelize the application and investigate performance by following these steps:
* Enable OpenMP in the compilation
* Parallelize the stream benchmark
    * Verify that the result is still correct
* Investigate the effect of applying different schedules
    * Either edit the code or use the runtime schedule
    * Document your findings
* Which configuration yields the highest performance?
* Is the observed bandwidth stable across iterations?

You will likely see different effects that, depending on you background, may not match your expectations.
We will continue covering some of them in the [performance notebook](performance.ipynb).

In [ ]:
!g++ -O3 -std=c++17 -Wall -o code/examples/stream code/examples/stream.cpp

In [ ]:
!code/examples/stream $((32 * 1024 * 1024)) 8

### Solution

You can find one possible solution at [code/solutions/loops/stream.cpp](code/solutions/loops/stream.cpp).
The following cells evaluate different scheduling strategies.

In [ ]:
!g++ -O3 -std=c++17 -Wall -fopenmp -o code/solutions/loops/stream code/solutions/loops/stream.cpp

In [ ]:
!OMP_SCHEDULE=static code/solutions/loops/stream $((32 * 1024 * 1024)) 8

In [ ]:
!OMP_SCHEDULE=guided,1024 code/solutions/loops/stream $((32 * 1024 * 1024)) 8

In [ ]:
!OMP_SCHEDULE=dynamic,16384 code/solutions/loops/stream $((32 * 1024 * 1024)) 8

## Nested Loops

OpenMP allows collapsing multiple nested loops into one logical iteration space with the `collapse(n)` clause where `n` is the number of loops to collapse.
This can be useful when
* the outer loop length is not sufficiently large to satisfy the available number of threads,
* or in case of load imbalances,
* or for offloading work to a [target](target-offloading.ipynb).

If no collapse clause is specified, an implicit `collapse(1)` is assumed.

### Base Case

In [ ]:
%%cpp_omp -o code/loops/nested.cpp -t

constexpr auto N = 4, M = 4;

#pragma omp parallel for num_threads(8) // collapse(1)
for (auto i = 0; i < N; ++i) {
    for (auto j = 0; j < M; ++j) {
        std::this_thread::sleep_for(std::chrono::milliseconds(j * N + i));
        std::cout << "i = " << i << ", j = " << j << ", threadId = " << omp_get_thread_num() << std::endl;
    }
}

### Collapse with Default Scheduling

In [ ]:
%%cpp_omp -o code/loops/collapse.cpp -t

constexpr auto N = 4, M = 4;

#pragma omp parallel for num_threads(8) collapse(2)
for (auto i = 0; i < N; ++i) {
    for (auto j = 0; j < M; ++j) {
        std::this_thread::sleep_for(std::chrono::milliseconds(j * N + i));
        std::cout << "i = " << i << ", j = " << j << ", threadId = " << omp_get_thread_num() << std::endl;
    }
}

### Collapse with Dynamic Scheduling

In [ ]:
%%cpp_omp -o code/loops/collapse-dynamic.cpp -t

constexpr auto N = 4, M = 4;

#pragma omp parallel for num_threads(8) collapse(2) schedule(dynamic)
for (auto i = 0; i < N; ++i) {
    for (auto j = 0; j < M; ++j) {
        std::this_thread::sleep_for(std::chrono::milliseconds(j * N + i));
        std::cout << "i = " << i << ", j = " << j << ", threadId = " << omp_get_thread_num() << std::endl;
    }
}

### Restrictions

1. Loops must be perfectly nested.

In [ ]:
%%cpp_omp -o code/loops/wrong-nest.cpp -t

constexpr auto N = 4, M = 32;

#pragma omp parallel for num_threads(8) collapse(2) schedule(dynamic)
for (auto i = 0; i < N; ++i) {
    int someCode = 0;
    for (auto j = 0; j < M; ++j)
        std::this_thread::sleep_for(std::chrono::milliseconds(j * N + i));
}

2. Loops must form a rectangular iterations space.

In [ ]:
%%cpp_omp -o code/loops/non-rect.cpp -t

constexpr auto N = 16;

#pragma omp parallel for num_threads(8) collapse(2) schedule(dynamic)
for (auto i = 0; i < N; ++i) {
    for (auto j = 0; j < i; ++j)
        std::this_thread::sleep_for(std::chrono::milliseconds(j * N + i));
}

## lastprivate

Adding the `lastprivate` clause for a variable to a worksharing loop construct
* privatizes that variable, and
* stores the result of the *last iteration* in the original variable upon finishing the loop.

In [ ]:
%%cpp_omp -o code/loops/lastprivate.cpp

constexpr auto N = 1024;
int vec[N];

int last;

# pragma omp parallel for lastprivate(last)
    for (auto i = 0; i < 1024; ++i) {
        vec[i] = i;
        last = i;
    }

std::cout << last << " should be " << vec[N - 1] << std::endl;

## Loop Form

OpenMP requires a *canonical loop nest form* ([OpenMP 5.1 - 2.11.1](https://www.openmp.org/spec-html/5.1/openmpsu45.html)).
It is either
* `for (init-expr; test-expr; incr-expr)`, or
* `for (range-decl: range-expr)` (OpenMP $\ge$ 5.0)

with a number of restrictions.
Most notably

* loop iterators need to be of integer, pointer or random access iterator type,
* increments must be linear, and
* multiple loop counters are not supported.

### Valid Examples

In [ ]:
%%cpp_omp -o code/loops/loop-form-valid.cpp -i vector

std::vector<int> vec = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};

#pragma omp parallel for
for (size_t i = 0; i < vec.size(); ++i)
    vec[i] += 10;

#pragma omp parallel for
for (auto it = vec.begin(); it != vec.end(); ++it)
    *it += 10;

#pragma omp parallel for
for (auto& val : vec)
    val += 10;

for (const auto& val : vec)
    std::cout << val << " ";
std::cout << std::endl;

### Invalid Examples

In [ ]:
%%cpp_omp -o code/loops/loop-form-invalid.cpp -i vector

std::vector<int> vec = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};

//#pragma omp parallel for
//for (float i = 0; i < vec.size(); ++i)
//    vec[i] += 10;

//#pragma omp parallel for
//for (size_t i = 1; i < vec.size(); i *= 2)
//    vec[i] += 10;

//auto test = [&](size_t i){return i < vec.size();};
//#pragma omp parallel for
//for (size_t i = 0; test(i); ++i)
//    vec[i] += 10;

//#pragma omp parallel for
//for (size_t i = 0, j = 1; i < vec.size(); ++i, ++j)
//    vec[i] += 10;

for (const auto& val : vec)
    std::cout << val << " ";
std::cout << std::endl;

## Exercise: 2D Stencil Benchmark

<div class="alert alert-block alert-success"> <b>Exercise:</b> Investigate different loop parallelizations. </div>

Check the code for the 2D stencil application at [code/examples/stencil-2d.cpp](code/examples/stencil-2d.cpp) and the documentation in the [examples notebook](examples.ipynb#2D-Stencil).
For convenience, the cells for building and executing are copied below.
\
Parallelize the application and investigate performance by following these steps similar to the last exercise:
* Enable OpenMP in the compilation
* Parallelize the stencil code
    * Verify that the result match those of the serial version
* Investigate the effect of different loop parallelizations
    * Outer loop parallelization
    * Inner loop parallelization
    * Collapsed loop parallelization
* Which configuration yields the highest performance?

As in the last exercise, not all effects might be fully understandable yet and some of them will be covered in the [performance notebook](performance.ipynb).

In [ ]:
!g++ -O3 -std=c++17 -Wall -o code/examples/stencil-2d code/examples/stencil-2d.cpp

In [ ]:
!code/examples/stencil-2d 4096 4096 64

### Solution

You can find possible solutions at
* [code/solutions/loops/stencil-2d-outer.cpp](code/solutions/loops/stencil-2d-outer.cpp),
* [code/solutions/loops/stencil-2d-inner.cpp](code/solutions/loops/stencil-2d-inner.cpp), and
* [code/solutions/loops/stencil-2d-collapse.cpp](code/solutions/loops/stencil-2d-collapse.cpp).

In [ ]:
!g++ -O3 -std=c++17 -Wall -fopenmp -o code/solutions/loops/stencil-2d-outer code/solutions/loops/stencil-2d-outer.cpp
!code/solutions/loops/stencil-2d-outer 4096 4096 64

In [ ]:
!g++ -O3 -std=c++17 -Wall -fopenmp -o code/solutions/loops/stencil-2d-inner code/solutions/loops/stencil-2d-inner.cpp
!code/solutions/loops/stencil-2d-inner 4096 4096 64

In [ ]:
!g++ -O3 -std=c++17 -Wall -fopenmp -o code/solutions/loops/stencil-2d-collapse code/solutions/loops/stencil-2d-collapse.cpp
!code/solutions/loops/stencil-2d-collapse 4096 4096 64

## Exercise: Julia Set

<div class="alert alert-block alert-success"> <b>Exercise:</b> Investigate different loop parallelizations. </div>

Check the code for the Julia set application at [code/examples/julia-set.cpp](code/examples/julia-set.cpp) and the documentation in the [examples notebook](examples.ipynb#Julia-Set).
For convenience, the cells for building and executing are copied below.
\
Parallelize the application and investigate performance by following these steps similar to the last exercise:
* Enable OpenMP in the compilation
* Parallelize the main loop
    * Verify that the result match those of the serial version
* Investigate the effect of different loop parallelizations and schedules
* Which configuration yields the highest performance?

In [ ]:
!g++ -O3 -std=c++17 -Wall -o code/examples/julia-set code/examples/julia-set.cpp code/examples/lodepng.cpp

In [ ]:
!code/examples/julia-set

The output image can be displayed using this cell.

In [ ]:
from IPython.display import display, Image
display(Image(filename='julia.png', width=512, height=512))

### Solution

You can find possible solutions at [code/solutions/loops/julia-set.cpp](code/solutions/loops/julia-set.cpp).

In [ ]:
!g++ -O3 -std=c++17 -Wall -o code/solution/loops/julia-set code/solution/loops/julia-set.cpp code/examples/lodepng.cpp
!OMP_SCHEDULE=static code/solution/loops/julia-set
!OMP_SCHEDULE=dynamic code/solution/loops/julia-set
!OMP_SCHEDULE=guided code/solution/loops/julia-set